# Redes neuronales junto con NeuralODEs

Vamos a mezclar los ejemplos vistos anteriormente, para ello vamos a crear una red neuronal la cual también pase por una NeuralODE

In [ ]:
using Flux
using DifferentialEquations
using DiffEqFlux
using Plots
using Flux: train!
using Distributions

Comenzamos como simpre generando los dastos que vamos a utilizar

In [ ]:
function Noise_Sine(x)
    return sin(2π*x) + rand(Normal(0,0.1))
end;

In [ ]:
t_train = Float32.(hcat(-1:0.01:1...))
#t_train_normalized = Float32.((t_train .- mean(t_train)) ./ std(t_train))
y_train = Float32.(Noise_Sine.(t_train))
trange = t_train[1,:]
tspan = (t_train[1], t_train[end])

In [ ]:
scatter(t_train[1,:], y_train[1,:], label="training data", title="Sine function with noise")

In [ ]:
dudt = Chain(x -> 2π*cos.(2π.*x),
            Dense(1 => 30, relu),
            Dense(30 => 25, relu),
            Dense(25 => 1, tanh_fast))

A esta NN la vamos a hacer pasar por una NerualODE y extraemos los paráemtros de este modelo

In [ ]:
n_ode = NeuralODE(dudt, tspan, Tsit5(), saveat = trange, )
ps = Flux.params(n_ode)

Creamos una función para crear las predicciones de la NODE empezando en el tiempo inicial t0.

In [ ]:
t0 = Float32[0.0]
function predict_n_ode()
    n_ode(t0)
end

Creamos la función costo para esta red neruronal, igual que antes usamos mse

In [ ]:
function loss_node()
    pred = predict_n_ode()
    pred = vcat(pred.u...)
    return mean(abs2, pred .- y_train[1,:])
end

Veamos como se ve inicialente las predicciones de la red neuronal

In [ ]:
pred0 = predict_n_ode()
scatter(trange, pred0[1,:], label="initial prediction", title="Sine function with noise")
scatter!(trange, y_train[1,:], label = "training data")

Seteamos la cantidad de épocas a entrenar, el ratio de aprendizaje y el modelo de optimización que vamos a utilizar. Además creamos la función callback para obtener información de cada época de entrenamiento

In [ ]:
data = Iterators.repeated((), 1000)
learning_rate = 0.01
opt = ADAM(learning_rate)
iter = 0
losses = []
cb = function () #callback function to observe training
  global iter += 1
  actual_loss = loss_node()
  if(iter%100 == 0)
    cur_pred = predict_n_ode()
    println("Epoch: $iter | Loss: $actual_loss")
    pl = scatter(trange,y_train[1,:],label="data")
    scatter!(pl,trange,cur_pred[1,:],label="prediction")
    display(plot(pl))
  end
  push!(losses, actual_loss)
end

Flux.train!(loss_node, ps, data, opt, cb = cb)
